#### 201724500 심진섭 Machine learning assignment #2
## 1. Get dataset

>가장 먼저, pdf에서 제공받은 iris(붓꽃) dataset을 가져와 읽어온다.

이전 과제와는 다르게 iris data로 데이터가 고정되었다.
기계 학습에서 가장 유명한 데이터로, 아래와 같은 Attribute를 가진다.
- Sepal length (cm) : 꽃받침의 길이
- Sepal width (cm) : 꽃받침의 너비
- Petal length (cm) : 꽃잎의 길이
- Petal width (cm) : 꽃잎의 너비

위의 4가지 Attribute를 통해 총 150개의 Example을 아래 3가지 class에 대해 학습한다.
- Setosa : 부채 붓꽃
- Versicolour : 버시칼라 붓꽃
- Virginica : 버지니카 붓꽃

## 1-1. 왜 Pytorch를 사용했는가?
이번 과제에서는 Keras, tensorflow 등 여러 프레임워크를 사용할 수 있지만 나는 Pytorch를 사용해보기로 했다.  
아래와 같은 이유로 Pytorch를 선택하게 되었다.  
- 이전에 학습한 적이 있어 다른 프레임워크들에 비해 익숙하다.
- 문법의 형태가 기존 파이썬에서 크게 벗어나지 않아 학습이 쉽다.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as met

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

col_names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'species']
df = pd.read_csv('iris.data', names=col_names)
df.head(5)

,sepal-length,sepal-width,petal-length,petal-width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 2. Data preprocessing
먼저, 아래와 같이 데이터 전처리를 진행한다.
- Categorical data로는 학습 및 분류가 불가능하다.
  - 따라서 Numerical data로 factorize() 함수를 통해 변환한다.
- Pytorch를 통해 기계 학습을 진행하려면 array가 아닌 Tensor의 형태여야한다.
  - torch.from_numpy() 함수를 이용해 변환한다.
- Train 60%, Test 20%, Valid 20%로 dataset을 분할해야 한다.
  - Scikit learn 패키지의 train_test_split을 이용해서 random하게 분할한다.
    - 먼저 Train : Test를 60 : 40으로 분할한다.
    - 이후 Test : valid를 50 : 50으로 다시 분할한다.

아래와 같은 shape의 가공된 학습 데이터가 만들어진다.
- Train set : [90, 4]
- Test set : [30, 4]
- Valid set : [30, 4]

In [ ]:
# 우선 학습을 위해 Dataset을 입력과 결과로 자른다.
# 이 때, 학습을 위해 Categorical data로 되어있는 y를 Numerical 하게 변환해준다.
df['species'], _ = df['species'].factorize() # Categorical -> Numerical

x = df.iloc[:,0:4].values # 입력
y = df.iloc[:,4].values # 출력

# Pytorch를 통한 기계학습을 위해 Array를 Tensor로 바꾸어 준다.
x = torch.from_numpy(x).type(torch.float)
y = torch.from_numpy(y).type(torch.long)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_valid.shape, y_valid.shape)

torch.Size([90, 4]) torch.Size([90])
torch.Size([30, 4]) torch.Size([30])
torch.Size([30, 4]) torch.Size([30])


## 3. Neural Network Learning
이제 실제로 Feedforward Neural Netwrok를 구성하여 실제로 데이터를 학습시켜 볼 차례이다.  

### Network architecture
Layer를 쌓는 방식은 자유이므로, 나는 아래와 같이 간단한 network를 구성하였다.
- Fully connected layer (90 * 4 -> 90 * 256)
- ReLU (Activation function)
- Fully connected layer (90 * 256 -> 90 * 128)
- ReLU (Activation function)
- Fully connected layer (90 * 128 -> 90 * 3)  

최초에는 Layer의 크기를 16, 32와 같이 적게 시도했다.  
이후 64, 128, 256과 같이 점차 늘려갈 수록 더 높은 정확도를 보였다.  
하지만 512, 1024와 같이 너무 많은 neuron의 수는 오히려 정확도를 떨어트림을 확인할 수 있었다.  

### Optimizer
Optimizer은, weight를 어떤 방식으로 최적화 시켜나갈 것인지에 대한 설정이다.  
Stochastic Gradient Descent를 기본적으로 채택하며, hyperparameter에 대한 설정은 자유이므로, 아래와 같이 설정했다.
- learning rate(alpha) : 0.01[링크 텍스트](https://)

### Loss function
Loss function은 Categorical cross entropy 방식으로 고정인데,  
Keras의 Categorical cross entropy와 Pytorch의 nn.CrossEntropy는 동일한 기능을 제공한다.  

Epoch가 진행될 수록 낮아지는 loss를 확인할 수 있다.

In [ ]:
model = nn.Sequential(
    nn.Linear(4, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 3)
)
optimizer = optim.SGD(model.parameters(), lr = 0.01)
cross_entropy = nn.CrossEntropyLoss()

## Training 과정

for epoch in range(1000):
  optimizer.zero_grad() # 이전 epoch에서 쌓인 정보 초기화

  train_output = model(x_train) # Feedforward
  train_loss = cross_entropy(train_output, y_train) # Calculate loss
  train_loss.backward() # Backpropagation, gradient를 계산하는 것
  optimizer.step() # 위에서 얻은 정보를 통해 weight를 update한다.

  test_output = model(x_test) # Test set에 대한 feedforward
  test_loss = cross_entropy(test_output, y_test) # Test set의 loss

  valid_output = model(x_valid) # Valid set에 대한 feedforward
  valid_loss = cross_entropy(valid_output, y_valid) # Valid set의 loss

  print("======== Epoch %d =========" % (epoch + 1))
  print("Train loss : %f, Test loss : %f, Validation loss : %f" % (train_loss.item(), test_loss.item(), valid_loss.item()))
  print()

======== Epoch 1 =========
Train loss : 1.401608, Test loss : 1.082842, Validation loss : 1.054976

======== Epoch 2 =========
Train loss : 1.088606, Test loss : 1.017588, Validation loss : 1.027332

======== Epoch 3 =========
Train loss : 1.024620, Test loss : 0.983517, Validation loss : 1.012483

======== Epoch 4 =========
Train loss : 0.994837, Test loss : 0.958545, Validation loss : 0.997407

======== Epoch 5 =========
Train loss : 0.972710, Test loss : 0.934097, Validation loss : 0.976967

======== Epoch 6 =========
Train loss : 0.951309, Test loss : 0.913582, Validation loss : 0.959762

======== Epoch 7 =========
Train loss : 0.934203, Test loss : 0.894429, Validation loss : 0.944613

======== Epoch 8 =========
Train loss : 0.918484, Test loss : 0.875013, Validation loss : 0.929232

======== Epoch 9 =========
Train loss : 0.903805, Test loss : 0.859081, Validation loss : 0.913659

======== Epoch 10 =========
Train loss : 0.890083, Test loss : 0.842406, Validation loss : 0.899477


## 4. Scoring
분류에 대한 평가 지표로는 아래와 같은 지표들이 사용 가능하다.  

- Accuracy(정확도) : 정확히 예측한 Example 수 / 전체 Example 수
- Precision(정밀도) : 실제로 맞춘 Example 수 / 특정 Class라고 예측한 Example 수
  - 3가지 Class에 대해서 Average precision을 계산해서 도출한다.
- Recall(재현률) : 실제로 맞춘 Eaxmple 수 / 실제 특정 Class의 Example 수
  - 3가지 Class에 대해서 Average recall을 계산해서 도출한다.
- F-1 Score : 2 * (precision X recall) / (precision + recall) (조화 평균)
- Confusion matrix를 통해 한 눈에 확인이 가능하다.

In [ ]:
# Accuracy 측정
def get_result_class(pred):
  result = [] # 예측된 Class를 저장할 배열
  pred = pred.numpy()
  
  for example in range(len(pred)):
    result.append(np.argmax(pred[example]))
    # 가장 큰 값이 나온 index가, 즉 예측된 class가 된다.

  return result
  # 몇 개나 맞췄는지 정확도를 반환한다.

def print_score(org_set, pred_set, set_name):
  print("Accuracy of", set_name, "set : ", round(met.accuracy_score(org_set, pred_set) * 100, 3))
  print("Precision of", set_name, "set : ", round(met.precision_score(org_set, pred_set, average = 'weighted') * 100, 3))
  print("Recall of", set_name, "set : ", round(met.recall_score(org_set, pred_set, average = 'weighted') * 100, 3))
  print("F-1 Score of", set_name, "set : ", round(met.f1_score(org_set, pred_set, average = 'weighted') * 100, 3))
  print()
  print(met.confusion_matrix(org_set, pred_set))
  print("=================================")

with torch.no_grad():
# torch.no_grad() 함수를 통해, weight 업데이트 없이 테스트가 가능하다.
  train_pred = model(x_train)
  test_pred = model(x_test)
  valid_pred = model(x_valid)

train_result = get_result_class(train_pred)
test_result = get_result_class(test_pred)
valid_result = get_result_class(valid_pred)

print(model)
print()
print_score(y_train, train_result, "train")
print_score(y_test, test_result, "test")
print_score(y_valid, valid_result, "valid")

Sequential(
  (0): Linear(in_features=4, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=3, bias=True)
)

Accuracy of  train  set :  97.778
Precision of  train  set :  97.908
Recall of  train  set :  97.778
F-1 Score of  train  set :  97.774

[[27  0  0]
 [ 0 29  2]
 [ 0  0 32]]
Accuracy of  test  set :  96.667
Precision of  test  set :  97.143
Recall of  test  set :  96.667
F-1 Score of  test  set :  96.722

[[12  0  0]
 [ 0  6  0]
 [ 0  1 11]]
Accuracy of  valid  set :  100.0
Precision of  valid  set :  100.0
Recall of  valid  set :  100.0
F-1 Score of  valid  set :  100.0

[[11  0  0]
 [ 0 13  0]
 [ 0  0  6]]
